In [10]:
import pandas as pd

import allel

from utils.prepare_data import read_data, save_data

In [11]:
data = allel.read_vcf(snakemake.input["vcf"])
sample_size = len(data["samples"])*2

In [12]:
records = list()

for position, haps in zip(data['variants/POS'], data['calldata/GT']):
    genotypes = haps.ravel()
    records.append((position, genotypes.sum(), len(genotypes), 1))
    
sf_data = pd.DataFrame.from_records(records, columns=['position', 'x', 'n', 'folded'])
save_data(sf_data, snakemake.output["data"])

In [13]:
sfs = read_data(snakemake.input["sfs"])
sfs.loc[0, "num_sites"] = 0
sfs = sfs.loc[sfs.num_alternate < sample_size]
sfs.num_sites = sfs.num_sites/sfs.num_sites.sum()
sfs.to_csv(snakemake.output["sfs"], header=False, index=False, sep='\t')